# COE782- ML - Lista3 - E2

## Dados câncer próstata
Resolução Bonus

Aluno: Vivian de Carvalho Rodrigues

DRE:125228569 

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LinearRegression, Ridge, Lasso 

#importa os dados do arquivo .csv na mesma pasta deste arquivo
df = pd.read_csv('dados_prostata.csv', sep ='\s+')

In [2]:
df.index = np.arange(97)
df.head()

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa,train
0,-0.579818,2.769459,50,-1.386294,0,-1.386294,6,0,-0.430783,T
1,-0.994252,3.319626,58,-1.386294,0,-1.386294,6,0,-0.162519,T
2,-0.510826,2.691243,74,-1.386294,0,-1.386294,7,20,-0.162519,T
3,-1.203973,3.282789,58,-1.386294,0,-1.386294,6,0,-0.162519,T
4,0.751416,3.432373,62,-1.386294,0,-1.386294,6,0,0.371564,T


In [3]:
#dados numéricos para padronização (desconsiderou-se as colunas de indices, "Train" e  "lpsa")
#Considera-se somente os dados de entrada 
data = df.iloc[:,:8].to_numpy()

In [4]:
data.shape

(97, 8)

In [5]:
#a)padronização dos dados
scaler = StandardScaler()
scaler

StandardScaler()

In [6]:
datastd = scaler.fit_transform(data)
datastd.shape

(97, 8)

In [7]:
#dataframe com dados de entrada X padronizados

#inclui os dados y ('lpsa') originais
df_std_aux= pd.DataFrame(datastd, columns= ['lcavol', 'lweight', 'age', 'lbph', 'svi', 'lcp', 'gleason', 'pgg45']).merge(df['lpsa'],left_index= True, right_index = True)
#incluindo novamente as classes de treino e teste
df_std=df_std_aux.merge(df['train'],left_index= True, right_index = True)


In [8]:
#b)divisão dos dados em treino e teste
#parcela de treino
dfs_train = df_std.loc[df_std['train'] == 'T']


In [9]:
#parcela de teste
dfs_test = df_std.loc[df_std['train'] == 'F']


In [10]:
#organiza dados de entrada e saída do treinamento: 67 registros
X_train = dfs_train.iloc[:,:8].to_numpy()
y_train = dfs_train.iloc[:,8:9].to_numpy().reshape(67)

#organiza dados de entrada e saída de teste: 30 registros
X_test = dfs_test.iloc[:,:8].to_numpy()
y_test = dfs_test.iloc[:,8:9].to_numpy().reshape(30)

In [11]:
y_train.shape

(67,)

In [12]:
#Modelo Ridge

freg =0.345                 #fator de regularização - lambda do Bishop
ridge = Ridge(alpha=freg)   #modelo
ridge.fit(X_train, y_train)
ridge.intercept_, ridge.coef_

(2.4652375921213907,
 array([ 0.66795437,  0.26171698, -0.13837883,  0.20821166,  0.3014456 ,
        -0.27585641, -0.01781654,  0.25911689]))

In [13]:
#predicoes
predt = ridge.predict(X_train)     #predicao dados de treinamento

residuals = y_train - predt
residuals.shape

(67,)

In [14]:
# Bootstrap
n_bootstraps = 1000
coef_samples_ridge = []

rng = np.random.default_rng(seed=42)
for _ in range(n_bootstraps):
    resampled_residuals = rng.choice(residuals, size=len(residuals), replace=True)
    y_bootstrap = predt + resampled_residuals

    model_boot = Ridge(alpha=0.345)
    model_boot.fit(X_train, y_bootstrap)
    coef_samples_ridge.append(model_boot.coef_)

coef_samples_ridge = np.array(coef_samples_ridge)
coef_std_ridge = coef_samples_ridge.std(axis=0)

In [15]:
# Resultados
print("\nRIDGE:")
for i, col in enumerate(df_std.iloc[:,:8].columns):
    print(f"{col:10s} | Coef: {ridge.coef_[i]: .4f} | Desvio padrão (bootstrap): {coef_std_ridge[i]: .4f}")


RIDGE:
lcavol     | Coef:  0.6680 | Desvio padrão (bootstrap):  0.1129
lweight    | Coef:  0.2617 | Desvio padrão (bootstrap):  0.0882
age        | Coef: -0.1384 | Desvio padrão (bootstrap):  0.0917
lbph       | Coef:  0.2082 | Desvio padrão (bootstrap):  0.0956
svi        | Coef:  0.3014 | Desvio padrão (bootstrap):  0.1127
lcp        | Coef: -0.2759 | Desvio padrão (bootstrap):  0.1384
gleason    | Coef: -0.0178 | Desvio padrão (bootstrap):  0.1291
pgg45      | Coef:  0.2591 | Desvio padrão (bootstrap):  0.1370


In [16]:
#Lasso
lasso = Lasso(alpha=0.37, max_iter=10000)
lasso.fit(X_train, y_train)
y_pred = lasso.predict(X_train)
residuals = y_train - y_pred

# Bootstrap
n_bootstraps = 1000
coef_samples_lasso = []

rng = np.random.default_rng(seed=42)
for _ in range(n_bootstraps):
    resampled_residuals = rng.choice(residuals, size=len(residuals), replace=True)
    y_bootstrap = y_pred + resampled_residuals

    model_boot = Lasso(alpha=0.37, max_iter=10000)
    model_boot.fit(X_train, y_bootstrap)
    coef_samples_lasso.append(model_boot.coef_)

coef_samples_lasso = np.array(coef_samples_lasso)
coef_std_lasso = coef_samples_lasso.std(axis=0)

# Resultados
print("LASSO:")
for i, col in enumerate(df_std.iloc[:,:8].columns):
    print(f"{col:10s} | Coef: {lasso.coef_[i]: .4f} | Desvio padrão (bootstrap): {coef_std_lasso[i]: .4f}")


LASSO:
lcavol     | Coef:  0.4729 | Desvio padrão (bootstrap):  0.0949
lweight    | Coef:  0.0892 | Desvio padrão (bootstrap):  0.0145
age        | Coef:  0.0000 | Desvio padrão (bootstrap):  0.0035
lbph       | Coef:  0.0000 | Desvio padrão (bootstrap):  0.0008
svi        | Coef:  0.0000 | Desvio padrão (bootstrap):  0.0106
lcp        | Coef:  0.0000 | Desvio padrão (bootstrap):  0.0192
gleason    | Coef:  0.0000 | Desvio padrão (bootstrap):  0.0009
pgg45      | Coef:  0.0000 | Desvio padrão (bootstrap):  0.0047
